In [1]:
import sys
import os
import glob

package_path = "../../funROI"
sys.path.append(os.path.abspath(package_path))
import funROI

funROI.set_bids_data_folder("../data/bids")
funROI.set_bids_deriv_folder("../data/bids/derivatives/funROI")
funROI.set_bids_preprocessed_folder("../data/bids/derivatives/fmriprep")

# Turn off all warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
from funROI.analysis import OverlapEstimator
from funROI.froi import FROI

# Parcels Overlap

In [3]:
overlap_estimator = OverlapEstimator(task_frois=['Events', 'Language'])
overlap_parcels = overlap_estimator.compute_overlap()
overlap_parcels.to_csv("overlap_parcels.csv", index=False)

In [4]:
import pandas as pd 
import numpy as np
funroi_data = pd.read_csv("overlap_parcels.csv")    
spm_data = pd.read_csv("../data/spm_ss_results/overlap_parcels.csv").dropna()
spm_data['ROI1'] = np.round(spm_data['ROI1'])
spm_data['ROI2'] = np.round(spm_data['ROI2'])

In [5]:
merged = pd.merge(funroi_data, spm_data, left_on=['froi1', 'froi2'], right_on=['ROI1', 'ROI2'])
merged = merged[['froi1', 'froi2', 'overlap', 'Overlap']].rename(columns={
    'overlap': 'funROI_overlap', 'Overlap': 'spm_overlap'
    })
(merged['funROI_overlap'] - merged['spm_overlap']).max(), (merged['funROI_overlap'] - merged['spm_overlap']).min()

(np.float64(2.220446049250313e-16), np.float64(-3.3306690738754696e-16))

# fROIs Overlap

In [6]:
subjects = [
    "040",
    "057",
    "059",
    "056",
    "067",
    "068",
    "019",
    "070",
    "087",
    "078",
    "089",
    "076",
]

frois = [
    (
        "SWNloc",
        [
            FROI(
                localizer="S-N",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Language",
            ),
        ],
    ),
    (
        "EventsOrig",
        [
            FROI(
                localizer="Sem_photominusPerc_photo",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Events",
            ),
            FROI(
                localizer="Sem_sentminusPerc_sent",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Events",
            ),
            FROI(
                localizer="SemMinusPercBothMax",
                contrasts=[
                    "Sem_photominusPerc_photo",
                    "Sem_sentminusPerc_sent",
                ],
                conjunctionType="max",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Events",
            ),
        ],
    ),
]

In [7]:
overlap_estimator = OverlapEstimator(subjects=subjects, task_frois=frois)
overlap_parcels = overlap_estimator.compute_overlap()
overlap_parcels.to_csv("overlap_frois.csv", index=False)

In [8]:
import pandas as pd
import numpy as np

funroi_data = pd.read_csv("overlap_frois.csv")
funroi_data['localizer1'] = funroi_data['localizer1'].map({
    'SemMinusPercBothMax': 'Sem-Perc_Both',
    'Sem_photominusPerc_photo': 'Sem-Perc_Photo',
    'Sem_sentminusPerc_sent': 'Sem-Perc_Sent',
    'S-N': 'S-N'
})
funroi_data['localizer2'] = funroi_data['localizer2'].map({
    'SemMinusPercBothMax': 'Sem-Perc_Both',
    'Sem_photominusPerc_photo': 'Sem-Perc_Photo',
    'Sem_sentminusPerc_sent': 'Sem-Perc_Sent',
    'S-N': 'S-N'
})

spm_data = []
spm_results_dir = '../data/spm_ss_results/overlap_frois'
subjects = ['040', '057', '059', '056', '067', '068', '019', '070', '087', '078', '089', '076']
localizers = ['Sem-Perc_Both', 'Sem-Perc_Sent', 'Sem-Perc_Photo', 'S-N']
for subject in subjects:
    for i, localizer1 in enumerate(localizers):
        for localizer2 in localizers[i:]:
            if localizer1.split('_')[0] == localizer2.split('_')[0]:
                if localizer1 != 'S-N':
                    df1 = pd.read_csv(f"{spm_results_dir}/{subject}_{localizer1}_{localizer2}_1.csv")
                    df1['subject'] = subject
                    df1['localizer1'] = localizer1
                    df1['localizer2'] = localizer2
                    spm_data.append(df1)    
                    df2 = pd.read_csv(f"{spm_results_dir}/{subject}_{localizer1}_{localizer2}_2.csv")
                    df2['subject'] = subject
                    df2['localizer1'] = localizer1
                    df2['localizer2'] = localizer2
                    spm_data.append(df2)
            else:
                df = pd.read_csv(f"{spm_results_dir}/{subject}_{localizer1}_{localizer2}.csv")
                df['subject'] = subject
                df['localizer1'] = localizer1
                df['localizer2'] = localizer2
                spm_data.append(df)
print(len(spm_data))         
spm_data = pd.concat(spm_data).dropna()
spm_data['ROI1'] = np.round(spm_data['ROI1'])
spm_data['ROI2'] = np.round(spm_data['ROI2'])
spm_data['subject'] = spm_data['subject'].astype(int)

spm_data['key'] = spm_data.apply(lambda x: tuple(sorted([(x['localizer1'], x['ROI1']), (x['localizer2'], x['ROI2'])])), axis=1)
funroi_data['key'] = funroi_data.apply(lambda x: tuple(sorted([(x['localizer1'], x['froi1']), (x['localizer2'], x['froi2'])])), axis=1)

180


In [9]:
spm_data = spm_data[['subject', 'key', 'Overlap']].groupby(['subject', 'key']).mean().reset_index()
funroi_data = funroi_data[['subject', 'key', 'overlap']].groupby(['subject', 'key']).mean().reset_index()

In [10]:
merged = pd.merge(funroi_data, spm_data, on=['subject', 'key'], how='right')
merged = merged.rename(columns={'overlap': 'funROI_overlap', 'Overlap': 'spm_overlap'})

In [11]:
# first 10 of absolute differences
merged['diff'] = np.abs(merged['funROI_overlap'] - merged['spm_overlap'])
merged = merged.sort_values('diff', ascending=False)
merged.head(10)


,subject,key,funROI_overlap,spm_overlap,diff
3789,59,"((Sem-Perc_Sent, 9), (Sem-Perc_Sent, 9))",0.287879,0.276923,1.095571e-02
5338,70,"((S-N, 1), (Sem-Perc_Sent, 4))",0.656000,0.664000,8.000000e-03
3492,59,"((Sem-Perc_Both, 9), (Sem-Perc_Sent, 9))",0.292308,0.284615,7.692308e-03
2970,57,"((Sem-Perc_Sent, 1), (Sem-Perc_Sent, 1))",0.471910,0.471910,4.996004e-16
3542,59,"((Sem-Perc_Photo, 1), (Sem-Perc_Photo, 1))",0.471910,0.471910,4.996004e-16
5193,68,"((Sem-Perc_Photo, 8), (Sem-Perc_Photo, 8))",0.448598,0.448598,4.996004e-16
2271,56,"((Sem-Perc_Sent, 9), (Sem-Perc_Sent, 9))",0.338462,0.338462,4.996004e-16
8334,87,"((Sem-Perc_Sent, 7), (Sem-Perc_Sent, 7))",0.413043,0.413043,4.996004e-16
3690,59,"((Sem-Perc_Photo, 9), (Sem-Perc_Photo, 9))",0.461538,0.461538,4.996004e-16
3417,59,"((Sem-Perc_Both, 7), (Sem-Perc_Both, 7))",0.304348,0.304348,4.996004e-16
